# Set the JSBSIM_ROOT_DIR variable
Already set in .bashrc but not read for some reasons when launching the notebook from the session.

In [1]:
import os
os.environ['JSBSIM_ROOT_DIR'] = '/home/ubuntu/jsbsim-JSBSim-trusty-v2018a'
os.environ['OPENAI_LOGDIR'] = '/home/ubuntu/logs'
os.environ['OPENAI_LOG_FORMAT'] = 'stdout,tensorboard'

# Imports

In [2]:
import sys
print ('\n'.join(sys.path))

import gym
import numpy as np

import argparse

import jsbsim
import gym_jsbsim

from stable_baselines.ddpg.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.ddpg.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines import DDPG
from stable_baselines import logger


/home/ubuntu/src/cntk/bindings/python
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages
/home/ubuntu/gym-jsbsim
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions
/home/ubuntu/.ipython


# Logging

In [3]:
def loggingFunction(dict1, dict2):
    print(dict1["action"])
    print("\n-----\n")
    print(dict1["new_obs"])
    print("\n-----\n")
    print(dict1["reward"])

    return

# Necessary for it to generate tensorboard log files during the run.  Note that
# in the docker file I set OPENAI_LOGDIR and OPENAI_LOG_FORMAT environment variables
# to generate the kind of logs that I want.
logger.configure()

Logging to /home/ubuntu/logs


# Environment setup

In [4]:
env = gym.make('JSBSim-HeadingControlTask-Cessna172P-NoFG-v0')
env = DummyVecEnv([lambda: env])
env.reset()
env.render(mode="human")

<Figure size 600x1100 with 4 Axes>

# Train

In [ ]:
# the noise objects for DDPG
n_actions = env.action_space.shape[-1]
param_noise = None
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))

try:
    model = DDPG.load("model/ddpg_fg_heading", env=env, tensorboard_log="model/tensorboard/")
    model.set_env( env )

except ValueError:  # Model doesn't exist

    model = DDPG(MlpPolicy, env, verbose=1, param_noise=param_noise, action_noise=action_noise, normalize_observations=True, tensorboard_log="model/tensorboard/")

while True:
    #model.learn(callback=loggingFunction, total_timesteps=1e6)
    model.learn(total_timesteps=1e6)
    model.save("model/ddpg_fg_heading")

-------------------------------------
| obs_rms_mean           | 564      |
| obs_rms_std            | 52.8     |
| reference_Q_mean       | -0.701   |
| reference_Q_std        | 0.421    |
| reference_action_mean  | 0.215    |
| reference_action_std   | 0.508    |
| reference_actor_Q_mean | -0.686   |
| reference_actor_Q_std  | 0.415    |
| rollout/Q_mean         | -0.527   |
| rollout/actions_mean   | 0.0793   |
| rollout/actions_std    | 0.492    |
| rollout/episode_steps  | 52.7     |
| rollout/episodes       | 189      |
| rollout/return         | -11.6    |
| rollout/return_history | -9.08    |
| total/duration         | 63.2     |
| total/episodes         | 189      |
| total/epochs           | 1        |
| total/steps            | 9998     |
| total/steps_per_second | 158      |
| train/loss_actor       | 0.706    |
| train/loss_critic      | 0.00564  |
-------------------------------------

-------------------------------------
| obs_rms_mean           | 565      |
| obs_rms_s